In [2]:
from Source.load_data import *
from Hyperparameters.params_TNG_CV import *
from Source.constants import *

use_model, learning_rate, weight_decay, n_layers, k_nn, n_epochs, training, simsuite, simset, n_sims = params
simpath = simpathroot + simsuite + "/"+simset+"_"
path = simpath + str(2)+"/fof_subhalo_tab_033.hdf5"
tab, HaloMass, HaloPos, HaloVel, halolist = general_tab(path)

In [26]:

ind = 5

tab_halo = tab[tab[:,0]==ind][:,1:] #1upwards because we don't need the hosthalo id from original tab
tab_halo.shape

(6, 8)

In [27]:
# Consider only halos with at least one satellite galaxy (besides the central)
if tab_halo.shape[0]>1:

    # Otherwise, write the positions and velocities as the relative position and velocity to the host halo
    tab_halo[:,0:3] -= HaloPos[ind]
    tab_halo[:,-3:] -= HaloVel[ind]

    # Correct periodic boundary effects
    tab_halo[:,:3] = correct_boundary(tab_halo[:,:3])

    # If use velocity, compute the betrag of the velocities vector and create a new table with these values
    if use_vel:
        subhalovel = np.log10(np.sqrt(np.sum(tab_halo[:,-3:]**2., 1)))  # use this way in case you normalize velocities
        newtab = np.column_stack((tab_halo[:,:-3], subhalovel))
    else:
        newtab = tab_halo[:,:-3]

In [44]:
size = tab_halo.shape[0] 
A = torch.ones(size,size)-torch.eye(size,size)
edge_index = A.nonzero().t().contiguous()

In [48]:
# Take as global quantities of the halo the number of subhalos and the total stellar mass
u = np.zeros((1,2), dtype=np.float32)
u[0,0] = tab_halo.shape[0]  # number of subhalos
if not only_positions:
    u[0,1] = np.log10(np.sum(10.**tab_halo[:,3]))

# Create the graph of the halo
# x: features (includes positions), pos: positions, u: global quantity
graph = Data(x=torch.tensor(newtab, dtype=torch.float32), pos=torch.tensor(tab_halo[:,:3], dtype=torch.float32), y=torch.tensor(np.log10(HaloMass[ind]), dtype=torch.float32), u=torch.tensor(u, dtype=torch.float))